<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200">
</div>

# QEC Decoder Toolkit

Quantinuum offers a real-time hybrid compute capability for Quantum Error Correction (QEC) workflows. This capability executes [Web Assembly (Wasm)](https://webassembly.org/) in the H-Series stack and enables use of libraries (e.g. linear algebra and graph algorithms) and complex datastructures (e.g. vectors and graphs) during real-time execution of a quantum circuit. Unlocking experimentation of basic QEC workflows is an important milestone, as Quantinuum plans to upgrades its H-Series hardware and deploy practical QEC schemes at scale. Wasm has been used, previously, in quantum error correction schemes [[arxiv.2208.01863](https://arxiv.org/abs/2208.01863), [arxiv.2305.03828](https://arxiv.org/abs/2305.03828)]. Syndrome measurement results are sent to a classical computer where a decoding algorithm is used to determine recovery operations and update quantum circuits in real-time. 

Wasm is a binary instruction format. To compile Wasm binaries, a low-level programming language, such as Rust or C/ C++, is used to compose desired classical functions before compilation. The quantum computing workflow will then contain a call to Wasm. The folder this notebooks is in contains examples of this workflow. Wasm is used because it is a fast, safe, expressive, and easy to compile with familiar programming languages.

*[Setup and Workflow](#Setup-and-Workflow)* section includes setup instructions to setting up a python environment that can work with hybrid compute. Additional guidance is also provided on compiling Wasm binaries from Rust or C/C++ source. *[Basic Usage](#Basic-Usage)* details using TKET capabilities to link the Wasm module to a TKET circuit as well as submission to H-Series. The *[Repetition Code](#Repetition-Code)* section narrates how the Wasm feature can be used to enable a decoder-style lookup table in the Repetition code workflow.

<div style="text-align: center;">
    <img src="../figures/qec_workflow_schematic.png" width="1000">
</div>

The Mid-circuit Measurement and Reset (MCMR) with Qubit Reuse features are used in the Repetition Code workflow, which is also showcased [here](https://github.com/CQCL/pytket-quantinuum/blob/develop/examples/Quantinuum_mid_circuit_measurement.ipynb). An explanation of the H-Series submission workflow is found [here](https://github.com/CQCL/pytket-quantinuum/blob/develop/examples/Quantinuum_circuit_submissions.ipynb). The Wasm module is imported from TKET as well as several optional conditional operators. More information on the conditional operations available can be found in the TKET user manual ([Classical and Conditional Operations](https://tket.quantinuum.com/user-manual/manual_circuit.html#classical-and-conditional-operations)).


**Contents**
* [Setup and Workflow](#Setup-and-Workflow)
* [Basic Usage](#Basic-Usage)
* [Repetition Code](#Repetition-Code)
* [Summary](#Summary)


## Setup and Workflow


To create and use your own Hybrid Compute functions in your workflow, it is necessary to use Wasm and a language that compiles code to Wasm. These instructions contain the steps for setting up an environment to do this.

Wasm can be generated by composing Rust and C/ C++ code, followed by a compilation step with the Rust compiler or LLVM toolchain (Clang compiler). Once generated, the Wasm binary can be injected into the TKET workflow. The main benefit is to enable classical compute in the H-Series stack during real-time execution of the quantum circuit.

It is recommended to either use Rust or C++ for the compilation language to Wasm. The languages have functionality that easily compiles to Wasm. For other language options see [Wasm Getting Started Developer's page](https://webassembly.org/getting-started/developers-guide/).

<div style="text-align: center;">
    <img src="../figures/hybrid_compute_fig1.png" width="500">
</div>

The following subsections will go over:
1. Install either Rust or C/C++
1. Define the classical functions to use in your program
1. Compile to Wasm
1. Use Compiled Wasm

**Note:** This notebook uses existing Wasm examples provided in this folder's subdirectories. The subfolders are organized as follows.

---
    ├── Hybrid-compute              <- Examples of how to use Hybrid Compute via WASM
        |
        ├── repeat_until_success    <- Repeat Until Success source code
        ├── repetition_code         <- Repetition Code source code
---



### 1. Install Rust or C/C++

#### Rust

If you're new to Rust, see the [The Rust Programming Language](https://doc.rust-lang.org/book/title-page.html) book. Note that Cargo is the Rust package manager. For more info on Cargo, see [The Cargo Book](https://doc.rust-lang.org/cargo/guide/).

If using Visual Studio code, you can also install the `rust-analyzer` extension in the Extensions tab.

##### a. Install Rust

Install Rust using the [Rust Installer](https://www.rust-lang.org/tools/install).

**Windows Users:** Before running the installer, you'll need to install the [Microsoft C++ Build Tools](https://visualstudio.microsoft.com/visual-cpp-build-tools/).


##### b. Create a new Rust project

Navigate to wherever you plan to store your Rust project files, and run: `cargo new --lib rus_wasm`. The project name *rus_wasm* can be whatever project name you want to use.

This creates a new library in a subdirectory with the project name you used with everything you need to get going.

* The *src/lib.rs* file contains some generated Rust code. This code can be erased and replaced with code you plan to use in your hybrid compute program.
* Inside this subdirectory is a file called *cargo.toml*. This file will be used to configure your build.

#### C/C++

##### a. Install Clang

For Windows 10/11 users, instruction should be followed [here](https://learn.microsoft.com/en-us/cpp/build/clang-support-msbuild?view=msvc-170) to install clang.

For Ubuntu 20.04 users, the development tools group needs to be installed with Aptitude. The LLVM and Clang compilers need to be installed separately.
```
sudo apt-get install build-essential
sudo apt-get install clang
sudo apt install lld-14
```

MacOS users must install XCode command line tools to enable clang usage.
```
xcode-select --install
```

##### b. Create a new Clang project

Navigate to wherever you plan to store your C/ C++ project files. Create a src folder containing a *.c source file. The Wasm binary that is generated, exists one-level higher than the src directory.

### 2. Define classical functions in your language of choice

Define the classical functions you plan to use in your hybrid compute program. These will be the classical functions that your quantum program will run during execution. In Rust, edit the src/lib.rs file. For C/ C++, edit the src/lib.c file. 

Examples of the types of functions that can be put in Rust or C code are provided in the respective folders within each of the example folders. *We recommend looking at one of the provided examples to know which functions and attributes are required for Hybrid Compute to work.*

*Note:* The standard C/ C++ library cannot be used in a program that is intended to be compiled to Wasm.

### 3. Compile code to Wasm

##### Rust 

1. Configure the `cargo.toml` file. You can use one of the examples provided to modify yours. 
1. Navigate to your Rust project directory from the command line.
1. Run: `cargo build`. 
    - If successful, a `target` folder and `cargo.lock` file will be created, alongside the output of the command displaying "Finished."
    - If failed, Cargo will output a messaging related to the reason.
1. Run: `rustup target add wasm32-unknown-unknown`. This allows Rust to used Wasm as a compilation target. This only needs to be run once for your environment. If you have run this previously when compiling Rust to Wasm, you do not need to run it again. 
1. Run: `cargo build --release --target wasm32-unknown-unknown`. This compiles your Rust code to Wasm.


##### C/C++

The `C`/`C++` compilation uses the `LLVM` toolchain and requires that no standard headers be used within the program. The compilation can be invoked with the `clang` (`clang++`) compiler with specific options to compile `lib.c` (`lib.cpp`) into a Wasm binary, `lib.wasm`:

```clang --target=wasm32 --no-standard-libraries -Wl,--no-entry -Wl,--export-all -o lib.wasm lib.c```

The following compiler options are used upon Clang invocation:

* `--target=wasm32`: Specify the target output as 32 bit Wasm binary.
* `--no-standard-libraries`: A compiler option to prevent usage of `libc`.
* `-Wl,--no-entry`: A linker option that disables the check for a main function in the `c` source.
* `-Wl,--export-all`: 

### 4. Use compiled Wasm

In order to use the compiled Wasm from your quantum program, you will want to find the associated `.wasm` file that the compilation step created. With Rust, this can be found in the `./target/wasm32-unknown-unknown/release` folder of your project. You'll note a file with the name of your project with the extension `.wasm`. To use this in your quantum program, you can copy-paste it into another folder, or ensure that the file path for your Wasm file handler points to this location.

#### Wasm Guidelines

Some limitations exist on what kinds of functions or capabilities are enabled on Quantinuum hardware. 

1. Guidelines writing source code:
    * The `*.rs` or `*.c` source must contain an empty void method named `init`. This is to initialize the Wasm environment.
    * Examples of the types of functions that can be put into code are provided in the `c/src` or `rust/src` folders within each of the example folders.
    * The function to compile to Wasm must accept multiple integers and return exactly one integer.
    * Rust source code can use the Rust standard library.
    * C source code cannot use the C standard library, WASI is needed for this, which breaks the Wasm memory sandbox.
    * The following is not allowed:
        * Random number generation
        * File I/O
        * Exception Handling
1. Quantum programs can call two types of Wasm functions:
    * Void Wasm functions, e.g. to initialize the Wasm environment.
    * Wasm functions that accept multiple (>=0) integers and return one or none integer. For example, examine the call `x = wasm_func(a, b, c);` The `wasm_func` function can exist in QASM where `a`, `b`, `c` and `x` are classical registers. Such function calls have timing restrictions. Wasm calls in quantum programs have required multiple milliseconds.
1. The state of a Wasm program persists between calls in the quantum programs within the limits of a job chunk window (~300 shots). 
    * This means global variables can hold a mutable state that is modified between calls.
    * One can imagine using various global arrays. It's possible to have global pointers work with heap allocated data structures (for example using unsafe Rust features when compiling to Wasm).
1. Currently, the combined Quantum Web Assembly programs have to come in under 6 MB.

## Basic Usage

To inject Wasm into a TKET program and enable processing on the H-Series devices or emulators, three components must be incorporated in the workflow.

1. [pytket.wasm.WasmFileHandler](https://tket.quantinuum.com/api-docs/wasm.html#pytket.wasm.WasmFileHandler): Enables addition of the Wasm module to the TKET program. Additionally, checks Wasm binary is compatible with Wasm standards (number of input parameters and input/ output parameter types) via a boolean kwarg, `check_file`.
1. [pytket.circuit.Circuit.add_wasm_to_reg](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.add_wasm_to_reg): Adds the linked Wasm to the TKET program as a classical operation to act across specified registers.
1. [pytket.circuit.Circuit.add_wasm](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.add_wasm): Adds the linked Wasm to the TKET program as a classical operation to act on specific bits.
1. [wasm_file_handler](https://tket.quantinuum.com/extensions/pytket-quantinuum/api.html#pytket.extensions.quantinuum.QuantinuumBackend.process_circuits) kwarg on `QuantinuumBackend.process_circuits`: The linked Wasm module must be specified during job submission to H-Series.

Components 1 and 2 are used in the *Job Construction* subsection. Component 3 is used in the *Job Submission* subsection. 

Usage of Wasm is showcased in a Repeat Until Success (RUS) example. The RUS protocol, initially introduced in [arxiv.1311.1074](https://arxiv.org/abs/1311.1074), is a non-deterministic decomposition technique to approximate a 1-qubit unitary within an allowable error threshold. The RUS circuit is composed of multiple subcircuits and each subcircuit runs successively if a specific condition is satisfied.

### Circuit Construction

An example is illustrated adding Wasm operations to a `pytket.circuit.Circuit` instance. Each Wasm operation calls a classical function from the linked Wasm binary.

A 2-qubit circuit is initialized. Subsequently, the `add_c_register` is used to add 4 classical registers. The first argument defines the name of the classical register. The second argument defines the size of the classical register. The following classical registers are added manually:

* `creg0` with 1 bit
* `creg1` with 1 bit
* `cond` with 32 bits
* `count` with 32 bits

In [ ]:
from pytket.circuit import Circuit, Qubit
from pytket.circuit.logic_exp import reg_lt

circuit = Circuit(2, name=f"RUS Circuit")

# Add classical registers
creg0 = circuit.add_c_register("creg0", 1)
creg1 = circuit.add_c_register("creg1", 1)
cond = circuit.add_c_register("cond", 32)
count = circuit.add_c_register("count", 32)

The `pytket.wasm.WasmFileHandler` instance is used to link to the Wasm module.

In [ ]:
import pathlib
from pytket.wasm import WasmFileHandler

rus_dir = pathlib.Path().cwd().joinpath("repeat_until_success")
wasm_file = rus_dir.joinpath("rus.wasm")
wfh = WasmFileHandler(wasm_file)

print(repr(wfh))

The linked Wasm module is added to `pytket.circuit.Circuit` instance using `add_wasm_to_reg`. The Wasm method `add_count` is called and the bit registers `creg1` and `count` are supplied. The output of `add_count` is written to `cond`. All the operations in this block are conditional operations on the decimal value of the  bit register `cond`.

In [ ]:
n_repetitions = 5
cond_execute = 3

for loop_iter in range(1, n_repetitions + 1):
    circuit.H(0, condition=reg_lt(cond, cond_execute))
    circuit.CX(0, 1, condition=reg_lt(cond, cond_execute))
    circuit.Measure(Qubit(1), creg1[0], condition=reg_lt(cond, cond_execute))

    # Call Wasm
    circuit.add_wasm_to_reg(
        "add_count", wfh, [creg1, count], [cond], condition=reg_lt(cond, cond_execute)
    )

    circuit.add_c_setreg(loop_iter, count, condition=reg_lt(cond, cond_execute))
    circuit.Reset(0, condition=reg_lt(cond, cond_execute))

circuit.Measure(Qubit(0), creg0[0])
circuit.Measure(Qubit(1), creg1[0])

### Job Submission

A `QuantinuumBackend` instance is instantiated with the backend target specified as a H-Series emulator, `H2-1E`.

In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend("H2-1E")
backend.login();

The RUS circuit is compiled to the H-Series gate set.

In [ ]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=2)

In [ ]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(circuit)

The RUS circuit is submitted to the H-Series emulator using `process_circuits`. The `wasm_file_handler` keyword argument is used to submit the linked Wasm module to H-Series as well.

In [ ]:
handle = backend.process_circuit(compiled_circuit, n_shots=100, wasm_file_handler=wfh)

The status of the circuit can be queried with the `circuit_status` method.

In [ ]:
status = backend.circuit_status(handle)
print(status)
print(status.message)

Once the job result is ready `get_result` can be used to retrieve the data.

In [ ]:
result = backend.get_result(handle)

In [ ]:
distribution = result.get_counts(creg1)
logical_fidelity = distribution.get((0,)) / sum(distribution.values())
print(logical_fidelity)

## Repetition Code

The classical operations on bits and bit registers using `pytket.circuit.logic_exp` can be replaced with Wasm.

In [10]:
from pathlib import Path
from pytket.wasm import WasmFileHandler

wasm_file_path = Path().cwd().joinpath("repetition_code") / "rust" / "rc.wasm"
wasm_file_handler = WasmFileHandler(wasm_file_path, check_file=True)

Classical expressions can be applied to classical bits using the `pytket.circuit.logic_exp` submodule. These expressions perform boolean operations on classical bit registers.

The `QubitRegister` and `BitRegister` objects are used to allocate qubits and bits to the `pytket.circuit.Circuit` instance. The circuit contains two quantum registers, `q` and `a`, and three classical registers:

* `c`
* `pfu`
* `syn`

In [11]:
from pytket.circuit import Circuit, QubitRegister, BitRegister

circuit = Circuit()
circuit.name = "repetition-code"

qreg = QubitRegister("q", 3)
circuit.add_q_register(qreg)

areg = QubitRegister("a", 1)
circuit.add_q_register(areg)

creg = BitRegister("c", 3)
circuit.add_c_register(creg)

pfu = BitRegister("pfu", 3)
circuit.add_c_register(pfu)

syn = BitRegister("syn", 2)
circuit.add_c_register(syn);

The cell below adds the necessary operations to measure the stabilizers $\hat{Z}_0 \hat{Z}_1$ and $\hat{Z}_1 \hat{Z}_2$.

`pytket.circuit.logic_exp.RegXor` is used to perform the `XOR` operation on register `syn_old` and `syn`. This is added to the circuit with the method `add_classicalexpbox_register`. Finally the state of a classical register, `syn`, is copied to another classical register, `syn_old`. `syn_old[0]` and `syn_old[1]` are equal to `syn[0]` and `syn[1]`, respectively.

In [12]:
from pytket.circuit.logic_exp import RegXor

circuit.CX(qreg[0], areg[0])
circuit.CX(qreg[1], areg[0])
circuit.Measure(areg[0], syn[0])
circuit.Reset(areg[0])

circuit.CX(qreg[1], areg[0])
circuit.CX(qreg[2], areg[0])
circuit.Measure(areg[0], syn[1])
circuit.Reset(areg[0])

[CX q[0], a[0]; CX q[1], a[0]; Measure a[0] --> syn[0]; Reset a[0]; CX q[1], a[0]; CX q[2], a[0]; Measure a[0] --> syn[1]; Reset a[0]; ]

The `add_wasm_to_reg` method is used to apply the `decode3` function, defined in the Wasm module, to the bit registers, `syn` and `pfu`. The output of `decode3` is written to the bit register, `pfu`. `add_c_xor_to_registers` is used to set the value of `pfu` to the output of the expression, `pfu` ^ `pfu`. `add_c_setreg` is used to set the all the bits in the classical register, `pfu`, to `False`.

In [13]:
circuit.add_wasm_to_reg("decode3", wasm_file_handler, [syn], [pfu])
for i in range(3):
    circuit.X(qreg[i], condition_bits=[pfu[i]], condition_value=1);

One more round of Syndrome measurement is performed followed by another Wasm call.

In [14]:
circuit.CX(qreg[0], areg[0])
circuit.CX(qreg[1], areg[0])
circuit.Measure(areg[0], syn[0])
circuit.Reset(areg[0])

circuit.CX(qreg[1], areg[0])
circuit.CX(qreg[2], areg[0])
circuit.Measure(areg[0], syn[1])
circuit.Reset(areg[0])

circuit.add_wasm_to_reg("decode3", wasm_file_handler, [syn], [pfu])
for i in range(3):
    circuit.X(qreg[i], condition_bits=[pfu[i]], condition_value=1)

Three measurement operations are applied to qubits within `qreg`. The outcome of the measurements is stored in the corresponding bits within `creg`, i.e. a measurement on `qreg[0]` will have a measurement outcome stored in `creg[0]`. Three Measurement operations are added to each qubit in `qreg` and the corresponding bit in `creg`.

In [15]:
for i in range(3):
    circuit.Measure(qreg[i], creg[i]);

`add_classicalexpbox_bit`, `add_c_setbits`, `add_classicalexpbox_register` are used to perform additional operations on the classical registers. The `^` operation is also used instead of `RegXOR` to perform `XOR` on two classical registers. `add_classicalexpbox_bit` is used to perform operations on specific bits, whilst `add_classicalexpbox_register` allows classical operations on entire bit registers.

In [16]:
circuit.add_classicalexpbox_bit(creg[0] ^ creg[1], [syn[0]])
circuit.add_classicalexpbox_bit(creg[1] ^ creg[2], [syn[1]]);

In [17]:
circuit.add_classicalexpbox_register(pfu ^ creg, pfu);

In [ ]:
circuit.add_wasm_to_reg("reset", wasm_file_handler, [], []);

In [19]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(circuit)

The repetition code circuit with the Wasm operation can be submitted to the `H1-1E` emulator using the usual workflow: compilation, costing, submission, status check and, finally, job retrieval. The linked Wasm module must be specified during job submission.

In [20]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend(device_name="H1-1E")
backend.login();

In [21]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=2)
render_circuit_jupyter(compiled_circuit)

In [22]:
n_shots = 100

handle = backend.process_circuit(
    compiled_circuit, n_shots=n_shots, wasm_file_handler=wasm_file_handler
)

In [24]:
backend.circuit_status(handle)

CircuitStatus(status=<StatusEnum.ERROR: 'Circuit has errored. Check CircuitStatus.message for error message.'>, message='{"name": "repetition-code", "submit-date": "2024-07-30T17:28:47.333525", "result-date": null, "queue-position": null, "cost": "0", "error": {"code": 1000, "text": "1000: Compile error: [line:43 column:6] Unexpected token \'(\'"}}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [18]:
result = backend.get_result(handle)

In [19]:
result.get_distribution(creg)

{(0, 0, 0): 0.98, (0, 1, 0): 0.01, (1, 0, 0): 0.01}

The logical outcome is the parity of the data qubits after measurement and correction. For example, if the register `creg` is 111, then the raw outcome is 1^1^1 = 1.

1. If the correction is 000 (000^111 = 111), after correction the logical outcome is 1^1^1 = 1
2. If the correction is 010 (010^111 =  101), after correction the logical outcome is 1^0^1 = 0

If a logical zero state is prepared, then in the example above, the raw outcome is incorrect and the first corrected outcome is also incorrect (logical failure). The second corrected outcome is correct. 
 
My logical fidelity is, $\frac{N_{success}}{N_{shots}}$, where $N_{success}$ is the number of successfully corrected outcomes and $N_{shots}$ is the number of shots total.

In [20]:
import numpy as np
import pandas as pd

from pytket.backends.backendresult import BackendResult
from pytket.circuit import BitRegister


def logical_error(
    result: BackendResult,
    n_shots: int,
    classical_register: BitRegister,
    pauli_frame_register: BitRegister,
) -> pd.DataFrame:
    raw0_tot = 0
    cor0_tot = 0
    raw1_tot = 0
    cor1_tot = 0

    c_dist = result.get_distribution(classical_register)
    pfu_dist = result.get_distribution(pauli_frame_register)

    for c, p in zip(c_dist.keys(), pfu_dist.keys()):
        nc = len(c)
        cr1 = sum(c)
        cr0 = nc - cr1
        if cr1 < cr0:
            raw0_tot += 1
        elif cr1 > cr0:
            raw1_tot += 1

        np = len(p)
        pr1 = sum(p)
        pr0 = np - pr1

        if pr1 < pr0:
            cor0_tot += 1
        elif pr1 > pr0:
            cor1_tot += 1

    raw0_tot = raw0_tot / n_shots
    raw1_tot = raw1_tot / n_shots
    cor0_tot = cor0_tot / n_shots
    cor1_tot = cor1_tot / n_shots

    error0 = (cor0_tot * (1 - cor0_tot)) / n_shots
    error1 = (cor1_tot * (1 - cor1_tot)) / n_shots

    return pd.Series(
        [
            ["Raw Zeros", raw0_tot],
            ["Raw Ones", raw1_tot],
            ["Corrected Zeros", cor0_tot],
            ["Corrected Ones", cor1_tot],
            ["Error Corrected Zeros", error0],
            ["Error Corrected Ones", error1],
        ]
    )

In [21]:
logical_error(result, n_shots, creg, pfu).head()

0                                  [Raw Zeros, 0.03]
1                                    [Raw Ones, 0.0]
2                            [Corrected Zeros, 0.03]
3                              [Corrected Ones, 0.0]
4    [Error Corrected Zeros, 0.00029099999999999997]
dtype: object

## Summary

The hybrid compute feature enables classical compute logic compiled to a Wasm binary to be linked and added to a TKET program for processing on H-Series. The classical compute portion of the program is processed during real-time execution of the quantum circuit. This is the first major step to practical QEC at scale as H-Series hardware evolves towards fault-tolerance. The 3-qubit repetition code protocol uses Wasm to decode syndromes via a lookup table.

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>